In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import notebook_helper
import sys
sys.path.append('../experiments/')
results_dir = '../results/'

Load the results into a pandas dataframe

In [ ]:
dir_names = sorted([fname
                    for fname in os.listdir(results_dir)
                    if os.path.isdir(join(results_dir, fname))
                    and os.path.exists(join(results_dir, fname, 'results.pkl'))
                    ])
results_list = []
for dir_name in tqdm(dir_names):
    ser = pd.Series(
        joblib.load(join(results_dir, dir_name, 'results.pkl')))
    results_list.append(ser)

r = pd.concat(results_list, axis=1).T.infer_objects()
r.to_pickle(join(results_dir, 'results_aggregated.pkl'))

In [ ]:
r

In [ ]:
r.iloc[0]['resps']

In [ ]:
# fill missing args with default values from argparse
r = notebook_helper.fill_missing_args_with_default(r, fname='01_train_model.py')

# group using these experiment hyperparams when averaging over random seeds
group_keys = [k for k in notebook_helper.get_main_args_list(fname='01_train_model.py') if not k == 'seed']
ravg = (
    r
    .groupby(by=group_keys)
    .mean()
    .reset_index()
)

# apply cross validation
ravg_cv = (
    ravg
    .sort_values(by='accuracy_cv', ascending=False)
    .groupby(by=['model_name', 'dataset_name'])
    .first()
    .reset_index()
)
ravg_cv

In [ ]:
sns.barplot(data=ravg_cv, x='model_name', y='accuracy_test') #, hue='dataset_name')
plt.show()

Load an individual model

In [ ]:
run_args = r.iloc[0]
model = joblib.load(join(run_args.save_dir_unique, 'model.pkl'))
model